---
title: "Ribasim Delwaq coupling"
---

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import networkx as nx
from ribasim import Model
from ribasim.delwaq import generate

In order to generate the Delwaq input files, we need a completed Ribasim simulation (typically one with a results folder). Let's take a complete HWS model for example. Please adjust the paths and filenames to your own case.


In [ ]:
toml_path = Path("../../hws_2024_6_0/hws.toml")
output_path = Path(
    "../../hws_2024_6_0/delwaq"
)  # also set a path where we store the Delwaq input files
assert toml_path.is_file()

Now we can call `generate`, generating the required input files for Delwaq.

In [ ]:
graph, substances = generate(toml_path, output_path)

Let's inspect the generated files.


In [ ]:
list(output_path.iterdir())

The two output variables `graph` and `substances` are required for parsing the results of the Delwaq model later on. Nonetheless, we can also inspect them here, and inspect the graph.

In [ ]:
substances  # list of substances, as will be present in the Delwaq netcdf output

In [ ]:
# Let's draw the graph
# plt.clf()
plt.figure(figsize=(15, 15))
nx.draw(
    graph,
    pos={k: v["pos"] for k, v in graph.nodes(data=True)},
    with_labels=True,
    labels={
        k: v["id"] for k, v in graph.nodes(data=True)
    },  # uncomment for original node id
)

Let's create a geopackage that contains our node mapping between Ribasim and Delwaq.

In [ ]:
model = Model.read(toml_path)
nodes = model.node_table()

mapping = {k: v["id"] for k, v in graph.nodes(data=True)}

df = nodes.df[nodes.df["node_id"].isin(mapping.values())]
df.set_index("node_id", inplace=True)
df = df.loc[list(mapping.values())]
df["meta_delwaq_id"] = mapping.keys()

df.to_file(output_path / "mapping.gpkg")